#Skating

## install pyskl 

In [1]:
!git clone https://github.com/lbaf23/skating-pyskl.git && \
cd skating-pyskl && \
pip install -r requirements.txt && \
pip install -e .

Cloning into 'skating-pyskl'...
remote: Enumerating objects: 411, done.
remote: Counting objects: 100% (411/411), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 411 (delta 224), reused 407 (delta 220), pack-reused 0
Receiving objects: 100% (411/411), 1.23 MiB | 17.75 MiB/s, done.
Resolving deltas: 100% (224/224), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.6 MB 4.6 MB/s 
     |████████████████████████████████| 530 kB 56.0 MB/s 
     |████████████████████████████████| 60 kB 4.5 MB/s 
     |████████████████████████████████| 190 kB 74.2 MB/s 
  Created wheel for mmcv: filename=mmcv-1.5.0-py2.py3-none-any.whl size=807180 sha256=93bda1a54277fb43df11d435fdaccfa808844d41aa4b4f1e0168b7d812e4a6f4
  Stored in directory: /root/.cache/pip/wheels/09/b4/88/0fee9e050a776e49b7290c2853e8ec002eeb55d7f4a40cf78f
Successfully built mmcv
Looking in indexes: https://pypi.org/simple,

In [2]:
!pip install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 5.1 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=a0a0c99a687812e2659ba3d26c6a2632c422d8e1d8d50b12ef83ca04599063e1
  Stored in directory: /root/.cache/pip/wheels/46/20/07/7bb9c8c44e6ec2efa60fd0e6280094f53f65f41767ef69a5ee
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


##load dataset
> put `train_data.npy` and `test_data.npy` in your google drive dir `skating/`

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### transfer dataset
> generate .pkl file

In [4]:
import numpy as np
import os


def transfer_data(in_dir, out_dir):
    data_dir = os.path.join(in_dir, 'train_data.npy')
    label_dir = os.path.join(in_dir, 'train_label.npy')

    data = np.load(data_dir)  # 2294 3 2500 25 1
    label = np.load(label_dir, allow_pickle=True)  # 2294
    ans = {'split': {}, 'annotations': []}
    N, C, T, V, M = data.shape
    sep = int(N * 0.9)
    train = []
    val = []
    for i in range(0, N):
        if i < sep:
            train.append(str(i))
        else:
            val.append(str(i))
    split = dict(
        train=train,
        val=val
    )
    for i in range(data.shape[0]):
        frame = data[i]
        frame = np.transpose(frame, (3, 1, 2, 0))
        keypoint = frame[..., :2]
        keypoint_score = frame[..., 2]
        ans['annotations'].append({'frame_dir': str(i),
                                   'label': label[i],
                                   'img_shape': (1080, 720),
                                   'original_shape': (1080, 720),
                                   'total_frames': 2500,
                                   'keypoint': keypoint,
                                   'keypoint_score': keypoint_score})
    ans['split'] = split
    import pickle
    with open(os.path.join(out_dir, 'train.pkl'), 'wb') as f:
        pickle.dump(ans, f)

In [5]:
!mkdir skating-pyskl/data

In [6]:
transfer_data('drive/MyDrive/skating', 'skating-pyskl/data')

## train

In [7]:
!cd skating-pyskl && \
bash tools/dist_train.sh configs/stgcn/skating/j.py 1

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2981888/45929032 bytes (6.5%)7045120/45929032 bytes (15.3%)11272192/45929032 bytes (24.5%)15597568/45929032 bytes (34.0%)19775488/45929032 bytes